In [45]:

%pip install --quiet openai python-dotenv pandas itables




Note: you may need to restart the kernel to use updated packages.


In [46]:
%pip install --quiet langfuse

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import openai

# Wczytaj zmienne środowiskowe
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

vacation_emails = [
    "Witaj, Chciałbym pojechać na wakacje do Malagi na 7 dni. Mam już wybrany lot i nocleg. Uwielbiam organizować wszystko sam.",
    "Cześć, szukam agencji która pomoże mi zorganizować wyjazd do Włoch na 2 tygodnie.",
    "Hej, chcę pojechać na wakacje do Barcelony na 10 dni. Znasz jakieś dobre biuro do polecenia?",
    "Cześć, chcę pojechać na wakacje do Paryża na 3 dni. Czy możesz mi pomóc znaleźć oferty?",
    "Dawno nie byłem na wakacjach. Marzy mi się wyjazd do Aten na 2 tygodnie, ale nie mam czasu na organizację. Najchętniej skorzystałbym z pomocy biura podróży.",
]

def get_data_from_email(email, model="gpt-4o"):
    prompt = """ ... """  # <- (pozostaw cały prompt jak był)

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"```{email}```"},
    ]

    client = OpenAI()  # upewnij się, że masz openai>=1.0 zainstalowane

    response = client.chat.completions.create(
    model=model,
    messages=messages,
    )

result = response.choices[0].message.content


    try:
        return json.loads(result)
    except json.JSONDecodeError:
        return {"error": result}

def get_data_from_emails(emails):
    results = []
    for email in emails:
        data = get_data_from_email(email=email)
        results.append({"email": email, **data})
    return pd.DataFrame(results)


In [61]:
data =  get_data_from_emails(vacation_emails)
langfuse.flush()

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## Tworzenie datasetu

In [ ]:
dataset_name = "vacation-emails"
langfuse.create_dataset(name=dataset_name);

In [ ]:
expected_outputs = [
    {
    "city": "Malaga",
    "country": "Hiszpania",
    "duration": 7,
    "by_agency": False,
    "attractions": "Alcazaba, Castillo de Gibralfaro, Museo Picasso Malaga"
    },
    {
    "city": None,
    "country": "Włochy",
    "duration": 14,
    "by_agency": True,
    "attractions": "Koloseum, Fontanna di Trevi, Panteon"
    },
    {
    "city": "Barcelona",
    "country": "Hiszpania",
    "duration": 10,
    "by_agency": True,
    "attractions": "Sagrada Familia, Park Guell, La Rambla"
    },
    {
    "city": "Paryż",
    "country": "Francja",
    "duration": 3,
    "by_agency": True,
    "attractions": "Wieża Eiffla, Luwr, Katedra Notre-Dame"
    },
    {
    "city": "Ateny",
    "country": "Grecja",
    "duration": 14,
    "by_agency": True,
    "attractions": "Partenon, Akropol, Muzeum Akropolu"
    }
]


In [ ]:
for email, output in zip(vacation_emails, expected_outputs):
    langfuse.create_dataset_item(
      dataset_name=dataset_name,
      input=email,
      expected_output=output,
)

## Uruchamianie eksperymentów na datasecie

In [ ]:
def output_evaluator(expected_output, output):
    score = 0.0
    fields = ['city', 'country', 'duration', 'by_agency', 'attractions']
    for field in fields:
        if expected_output.get(field) == output.get(field):
            score += 1.0 / len(fields)
    return score

In [ ]:
dataset = langfuse.get_dataset(dataset_name)
models = ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini","gpt-3.5-turbo-0125"]

for model in models:
    for item in dataset.items:
        output, trace = get_data_from_email(email=item.input, model=model)

        item.link(
            trace,     
            run_name=f"eksperyment-z-modelem-{model}",
            run_description=f"Eksperyment z modelem {model}",
            run_metadata={"model": model},
        )

        # opcjonalnie możemy dodać funkcję oceniającą wynik 
        langfuse.score(
            trace_id=trace.id,
            name="extraction-score",
            value=output_evaluator(item.expected_output, output),
            comment="Jak dobrze llm radzi sobie z wyciągnięciem danych",
        )
langfuse.flush()

## Eksportowanie danych do dataframe

In [ ]:
traces = langfuse.fetch_traces()

In [ ]:
traces.data[0].json()

In [ ]:
data = []

for item in traces.data:
    data.append(
        {
            "timestamp": item.timestamp,
            "email": item.input[1]['content'],
            "city": item.output.get("city") if item.output else None,
            "country": item.output.get("country") if item.output else None,
            "duration": item.output.get("duration") if item.output else None,
            "by_agency": item.output.get("by_agency") if item.output else None,
            "attractions": item.output.get("attractions") if item.output else None,
            "error": item.output.get("error") if item.output else None,
            "latency": item.latency,
            "cost": item.total_cost,
            "html_path": item.html_path,
        }
    )


traces_df = pd.DataFrame(columns=['timestamp','email','city','country','duration','by_agency','attractions','error','latency','cost','html_path'],data=data)

In [ ]:
traces_df

## Alternatywa z dekoratorem

In [ ]:
# Importowanie modułów langfuse
from langfuse.decorators import observe
from langfuse.openai import OpenAI as LangfuseOpenAI


In [ ]:
# Z obserwacją
llm_client = LangfuseOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
@observe()
def get_data_from_email_observed(email,model="gpt-4o-mini"):

    prompt = """
    You are awesome helper in parsing the criteria for user vacation planning.
    You will be provided with contents of user emails that contain the following data:

    <destination> - the place the user wants to visit. It will be either a country or a city.
    <city> - this is taken from the <destination> and is the city the user wants to visit.
    <country> - this is taken from the <destination> and is the country the user wants to visit.
    <duration> - the number of days the user wants to stay in the <destination>. If the user will 
    provide the information in weeks or months then please convert the value to days.
    <by_agency> - this is tricky. From the provided information try to determine if the user wants to
    organize the trip by themselves or if they want to use a travel agency. 

    Return the value as a dictionary the following keys
    - city - as a string or null if not provided
    - country - as a string or null if not provided
    - duration - as an integer or null if not provided
    - by_agency - as a boolean or null if not provided

    For the information provided by the user create a recommendation for attractions that they should visit.
    Please add the information as a comma separated list of strings and put it in the "attractions" key in the dictionary.
    If the key does not exist then add it to the dictionary.
    Take your time to provide at most 3 best attraction based on the destination and the duration of the stay.

    The user will provide the data in polish and but you should respond with the json as defined above
    and you should use exactly those keys. Return a valid dictionary, nothing else.
    I will parse the result with json.loads() function in python so please make sure the result is valid.

    Here is an example of the email content:
    ```
    Witaj,
    Chciałbym pojechać na wakacje do Malagi na 7 dni. Czy możesz mi pomóc w znalezieniu oferty?
    ```

    In this case the dictionary should look like this:
    {
    "city": "Malaga",
    "country": "Hiszpania",
    "duration": 7,
    "by_agency": null
    }
    """

    messages=[
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": f"```{email}```",
        },
    ]

    chat_completion = llm_client.chat.completions.create(
        response_format={"type": "json_object"},
        messages=messages, 
        model=model,
        # dodatkowe
        name="get_data_from_email_observed",
    )
    resp = chat_completion.choices[0].message.content
    try:
        output = json.loads(resp)
    except:
        output = {"error": resp}
    return output


def get_data_from_emails_observed(emails):
    results = []
    for email in emails:
        results.append({"email" : email, **get_data_from_email_observed(email)})
    return pd.DataFrame(columns=['email','city','country','duration','by_agency','attractions','error'],data=results)

In [ ]:
get_data_from_emails_observed(vacation_emails)

In [ ]:
dataset = langfuse.get_dataset(dataset_name)
models = ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini","gpt-3.5-turbo-0125"]

for model in models:
    for item in dataset.items:

        with item.observe(
            run_name=f"eksperyment-z-modelem-{model}-ale-z-dekoratorem",
            run_description=f"Eksperyment z modelem {model} ale z dekoratorem",
            run_metadata={"model": model},
        ) as trace_id:

            output = get_data_from_email_observed(email=item.input, model=model)

            # opcjonalnie możemy dodać funkcję oceniającą wynik 
            langfuse.score(
                trace_id=trace_id,
                name="extraction-score",
                value=output_evaluator(item.expected_output, output),
                comment="Jak dobrze llm radzi sobie z wyciągnięciem danych",
            )
langfuse.flush()